In [1]:
import json
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import pickle   
from lifelines import CoxPHFitter # install lifelines-0.30.0 if not already installed
from lifelines.utils import concordance_index


In [3]:
# load test features
test_radiomics = pd.read_csv('D:\\Shruti\\IS_project\\test_radiomics_CAC_scores.csv')

# retain patient_id separately; you need the surival data for these patients also (survival time and event columns)
pids = test_radiomics['patient_id'].values
# survival_time = test_radiomics['survival_time'].values
# event = test_radiomics['event'].values
test_radiomics = test_radiomics.drop(columns=['patient_id'])

# remove features that dont have 'CA_score' in their name
keep_cols = [col for col in test_radiomics.columns if ('CA_score' in col)] # texture features
test_radiomics = test_radiomics[keep_cols]  

# add relevant suffix to each column name, acc. to the ROI (check results.json file used)
test_radiomics.columns = [col for col in test_radiomics.columns]
test_radiomics.head()

,CA_score
0,682.947594
1,4042.282104
2,2534.311222
3,1189.195496
4,659.855530


In [5]:
results_json = json.load(open('D:\\Shruti\\IS_project\\models\\CAC_scoring\\results.json'))
selected_features = results_json['selected_features']

# make new dataframe X_test from test_radiomics and retain patient_id_NSCLC column
X_test = test_radiomics.copy()

# filter only texture features (remove shape features)
f_names = X_test.columns.tolist()
cac_score_features = [f for f in f_names if 'CA_score' in f]
X_test = X_test[cac_score_features]

# scale test data using the same scalar used for training
with open('D:\\Shruti\\IS_project\\models\\CAC_scoring\\scalar.pkl', 'rb') as f:
    scaler = pickle.load(f)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)  


# select only the selected features from test_radiomics
X_test_selected = X_test[selected_features]

# check if the number of selected features matches the number of columns in X_test_selected
print(len(selected_features), len(X_test_selected.columns))

1 1


c:\Users\P70073858\.conda\envs\shruti_env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
# load CoxPH model from pickle file


with open('D:\\Shruti\\IS_project\\models\\CAC_scoring\\coxph_model.pkl', 'rb') as f:
    cph = pickle.load(f)

# predict risk scores for test set
risk_scores = -cph.predict_partial_hazard(X_test_selected)

# calculate C-index --> please ensure you pass the correct survival time and event data for the test set patients here
test_c_index = concordance_index(event_observed=survival_time_column,  # --> edit as needed
                                 predicted_scores=risk_scores,
                                 event_times=event_column) # --> edit as needed
print("C-index on test set: ", test_c_index)

NameError: name 'survival_time_column' is not defined